In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
REPRODUCTION_DATA_PATH = '../reproduction/'
athene_results = pd.read_csv(os.path.join(REPRODUCTION_DATA_PATH, \
     'athene', 'official_FNC_submission.csv'))
talos_results = pd.read_csv(os.path.join(REPRODUCTION_DATA_PATH, \
     'talos', 'official_averaged_2models_cor4.csv'))
ucl_results = pd.read_csv(os.path.join(REPRODUCTION_DATA_PATH, \
     'ucl', 'official_predictions_test_with_headlines.csv'))

In [3]:
bert_entailment_filename = 'comp_autosave_after_epoch_10'
bert_entailment_results = pd.read_csv(bert_entailment_filename + '.csv')
bert_entailment_results = bert_entailment_results.rename(columns=dict({'Stance': 'BertStance'}))
bert_entailment_results = bert_entailment_results.drop(['articleBody', 'Unnamed: 0'], axis=1)
bert_entailment_results = bert_entailment_results.drop_duplicates(subset=['Headline', 'Body ID'])

In [11]:
def combineResults(team_results, bert_entailment_results):

    team_results = team_results.rename(columns=dict({'Stance': 'TeamStance'}))
    formatted_results = pd.merge(team_results, bert_entailment_results, \
        how='left', on=['Headline', 'Body ID']
    )
    formatted_results['Stance'] = np.where(
        (formatted_results['TeamStance'] != 'unrelated') \
            & (~(formatted_results['Guess'].isnull())),
        formatted_results['Guess'], formatted_results['TeamStance']
    )
    #return formatted_results[['Headline', 'Body ID', 'Stance']]
    return formatted_results


In [5]:
bert_entailment_with_athene_results = combineResults(athene_results, \
     bert_entailment_results)
bert_entailment_with_athene_results.to_csv(
    bert_entailment_filename + '_athene.csv', index=False)

In [6]:
bert_entailment_with_talos_results = combineResults(talos_results, \
    bert_entailment_results)
bert_entailment_with_talos_results.to_csv(
    bert_entailment_filename + '_talos.csv', index=False)

In [7]:
bert_entailment_with_ucl_results = combineResults(ucl_results, \
    bert_entailment_results)
bert_entailment_with_ucl_results.to_csv(
    bert_entailment_filename + '_ucl.csv', index=False)